# Programación Entera: Heurísticas

## Contenidos
  
  * [Problema de Transporte](#Problema-de-Transporte)
  * [Problema de Asignación](#Problema-de-Asignación)
  * [Problema de la Mochila](#Problema-de-la-Mochila)
    * [Mochila Simple](#Mochila-Simple)
    * [Mochila Múltiple](#Mochila-Múltiple)
    * [Mochila Multidimensional](#Mochila-Multidimensional)

In [1]:
library(lpSolve)
library(magrittr)

## Funciones de Apoyo

In [2]:
dataset.folder <- "./data"

In [3]:
ReadDataSet <- function() {
    
}

In [4]:
Solve <- function(direction = "max", f.obj, f.con, f.rhs, f.dir = rep("<=", nrow(f.con)), ...) {
    lp(direction, f.obj, f.con, f.dir, f.rhs,...)
}

## Problema de Transporte

#### Descripción:
TODO

#### Modelo:
TODO

In [5]:
TransportationExact <- function(demand, capacity, cost, ...) {
    m <- length(capacity)
    n <- length(demand)    
    
    conditions <- matrix(0, nrow = n + m, ncol = n * m)
    for (i in 1:m) {
        conditions[i, seq(n) + (i - 1) * n] <- 1
    }
    for (i in (m+1):(n+m)) {
        conditions[i, seq(i - m, m * n, by = n)] <- 1
    }
    res <- c(rep("<=", m), rep(">=", n))
    return(Solve("min", cost, conditions, c(capacity, demand), res, ...))
}

In [6]:
transportation.problem.1 <- list(
    demand = c(45, 20, 30, 30),
    capacity = c(35, 60, 48),
    cost = matrix(c(8, 6, 10, 9,
                    9, 12,13, 7,
                    14,9, 16, 5), nrow = 3, byrow = TRUE)
)

In [7]:
a <- TransportationExact(transportation.problem.1$demand,
                    transportation.problem.1$capacity,
                    transportation.problem.1$cost)

matrix(a$solution, 
       nrow = 3, byrow =TRUE)
print(a$objval)

35,0,0,0
10,20,12,0
0,0,18,30


[1] 976


#### Heurísticas:
TODO

In [8]:
TransportationGreedy <- function(demand, capacity, cost) {
    m <- length(capacity)
    n <- length(demand) 
    sol <- list()
    
    sol$solution <- matrix(0, nrow = m, ncol = n)
    sol$objval <- 0
    while (!all(demand == 0)) {
        ij <- which(cost == min(cost[capacity > 0, demand > 0]), arr.ind = TRUE)
        ij.slice <- (which(capacity[ij[, 1]] > 0 & demand[ij[, 2]] > 0))[1]
        i <- ij[ij.slice, 1]
        j <- ij[ij.slice, 2]
        inc <- min(capacity[i], demand[j])    
        
        sol$solution[i, j] <- sol$solution[i, j] + inc
        capacity[i] <- capacity[i] - inc
        demand[j] <- demand[j] - inc
        sol$objval <- sol$objval + cost[i, j] * inc
    }
    return(sol)
}

In [9]:
a <- TransportationGreedy(transportation.problem.1$demand,
                    transportation.problem.1$capacity,
                    transportation.problem.1$cost)

matrix(a$solution, 
       nrow = dim(transportation.problem.1$cost)[1], 
       ncol = dim(transportation.problem.1$cost)[2])
print(a$objval)

15,20,0,0
30,0,30,0
0,0,0,30


[1] 1050


## Problema de Asignación

#### Descripción:
TODO

#### Modelo:
TODO

In [10]:
AssignmentExact <- function(cost, ...) {
    return(TransportationExact(rep(1, dim(cost)[1]), rep(1, dim(cost)[2]), cost))
}

In [11]:
(cost <- as.matrix(read.csv("./data/assignment/matriz_15x15_1.csv", header = FALSE)))

V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15
58,66,6,77,7,19,44,22,9,90,86,11,61,11,26
56,78,12,30,89,20,62,14,82,76,70,100,59,57,42
93,80,90,77,71,96,55,25,46,57,16,30,91,55,0
85,48,55,36,91,10,31,98,42,84,98,91,76,93,10
62,85,32,56,21,56,78,48,49,66,0,39,39,6,53
44,64,25,47,56,63,80,35,22,70,100,96,32,84,70
50,23,8,90,23,51,98,94,33,84,95,41,77,33,30
9,32,100,36,3,21,66,19,91,48,81,57,86,96,35
59,62,12,98,93,38,59,93,18,21,59,87,76,9,82
37,1,13,29,85,18,29,20,54,94,30,1,0,67,51


In [12]:
a <- AssignmentExact(as.matrix(cost))
matrix(a$solution, 
       nrow = dim(cost)[1], 
       ncol = dim(cost)[2], 
       byrow = TRUE)
a$objval

0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


[1] 135

#### Heurísticas:
TODO

In [13]:
AssignmentGreedy <- function(cost) {    
    return(TransportationGreedy(rep(1, dim(cost)[1]), rep(1, dim(cost)[2]), cost))
}

In [14]:
a <- AssignmentGreedy(as.matrix(cost))
matrix(a$solution, 
       nrow = dim(cost)[1], 
       ncol = dim(cost)[2], 
       byrow = TRUE)
a$objval

0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


V11 
218

## Problema de la Mochila

#### Descripción:
TODO

### Mochila Simple

#### Descripción:
TODO

#### Modelo:
TODO

In [15]:
KnapsackSimpleExact <- function(object, capacity) {
    
}

#### Heurísticas:
TODO

In [16]:
KnapsackSimpleGreedy <- function(object, capacity) {
    
}

### Mochila Múltiple

#### Descripción:
TODO

#### Modelo:
TODO

In [17]:
KnapsackMultipleExact <- function(object, capacity) {
    
}

#### Heurísticas:
TODO

In [18]:
KnapsackMultipleGreedy <- function(object, capacity) {
    
}

### Mochila Multidimensional

#### Descripción:
TODO

#### Modelo:
TODO

In [19]:
KnapsackMultidimensionalExact <- function(object, capacity, ...) {
    n <- length(object$value)
    m <- length(capacity)    
    return(Solve("max", object$value, t(object$weight), capacity, ...))
}

In [20]:
knapsack.multidimensional.problem.1 <- list(
    object = list(
        value = c(12, 13, 15, 10),
        weight = matrix(c(0.5, 0.3, 0.2,
                          1.0, 0.8, 0.2,
                          1.5, 1.5, 0.1,
                          3.1, 2.5, 0.4), nrow = 4, byrow = TRUE) 
    ),
    capacity = c(3.1, 2.5, 0.4)
)

In [21]:
a <- KnapsackMultidimensionalExact(
    knapsack.multidimensional.problem.1$object, 
    knapsack.multidimensional.problem.1$capacity,
    all.int = TRUE
)
matrix(a$solution, nrow = 1, byrow = TRUE)
a$objval

0,1,1,0


[1] 28

#### Heurísticas:
TODO

In [22]:
KnapsackMultidimensionalGreedy <- function(object, capacity) {
    
}